# Multiple Parameter Estimations
It is often desirable to run a single optimization problem many times. In copasi this is fairly easy using the `repeat` scan item. In Pycotools we can use the `MultipleParameterEstimation` class to the same end.   

## Pre-requisites
### Get the Demonstration model
-  Zi et al 2012

In [2]:
%matplotlib inline
import site
site.addsitedir('//home/b3053674/Documents/pycotools')
from pycotools import model, tasks, viz, misc
from pycotools.Tests import test_models
import os
import pandas
from lxml import etree


import logging
logging.basicConfig(format = '%(levelname)s:%(message)s')
LOG=logging.getLogger()

## get string model from test_models
zi_model_string = test_models.TestModels().zi_model()

## get a working directory. Change this to change this to wherever you like
directory = r'/home/b3053674/Documents/pycotools/ZiModel'

## choose path to zi model
zi_path = os.path.join(directory, 'zi2012.cps')

##write model to file
with open(zi_path, 'w') as f:
    f.write(zi_model_string)
    
## check file exists
if not os.path.isfile(zi_path):
    raise Exception

zi = model.Model(zi_path)

//home/b3053674/Documents/pycotools/pycotools/viz.py:116: FutureWarning: The pandas.parser module is deprecated and will be removed in a future version. Please import from the pandas.io.parser instead
  from pandas.parser import CParserError


root:INFO:27:    Initializing pycotools
root:INFO:28:    Initializing logging System
root:INFO:29:    logging config file at: //home/b3053674/Documents/pycotools/pycotools/logging_config.conf


IOError: [Errno 2] No such file or directory: '/home/b3053674/Documents/pycotools/ZiModel/zi2012.cps'

### Generate Synthetic Data

In [ ]:
report= 'parameter_estimation_synthetic_data.txt'
TC=tasks.TimeCourse(
    zi, start=0, end=1000, intervals=100, step_size=10, report_name=report
)

## validate that its worked
pandas.read_csv(TC.report_name,sep='\t').head()

## informative name
zi_data_file = TC.report_name

### Format synthetic data 

In [ ]:
misc.correct_copasi_timecourse_headers(zi_data_file)

# The MultipleParameterEstimation Class
The MultipleParameterEstimation class is an interface to the ParameterEstimation class and accepts all of the same keyword parameters with a few extra:

    1. `copy_number`          --> Number of times to copy a copasi file. All of these will be run simultaneously. 
    2. `pe_number`            --> How many parameter estimations to conduct **per model**
    3. `results_directory`    --> Name of the directory to put the results (created if not exists)

The total number of estimations that can be carried out simultaneously is $copy\_number\cdot pe\_number$. 

In [ ]:
MPE=tasks.MultiParameterEstimation(
    zi, zi_data_file, copy_number=8, pe_number=20, #--> 8 total
    metabolites=[], local_parameters=[], lower_bound=1e-3, upper_bound=5e3, 
    method='genetic_algorithm', population_size=10, number_of_generations=20)
    
MPE.write_config_file()
MPE.setup()
# MPE.run()

All models are executed using CopasiSE at the same time. For this reason, if you have too many models running at once on a single machine, the machine will slow and be unusable until the estimations have finished. My Windows machine (i7, 16GB RAM) can comfortably run around 6 or 7 models at once before things start to slow. My Ubuntu machine can run more but the estimations are significantly slower. If using a SunGridEngine based job scheduler you can use `run = SGE` to submit the jobs automatically. If your using a different job scheduler it is straight forward to write another function within the `pycopi.Run` class to support your scheduler. Feel free to contact me for advice.   

## Custom Results Directory

In [ ]:
results_directory = 'Fit1Results' #defaults to same directory as model
MPE = tasks.MultiParameterEstimation(
    zi, zi_data_file, copy_number=4, pe_number=2,
    results_directory=results_directory)
MPE.write_config_file()
MPE.setup()
MPE.run()

# Visualization
Exploring parameter estimation data is one of pycotools strengths. 

## Time course ensembles

In [ ]:
viz.EnsembleTimeCourse(MPE)

## Boxplots

In [ ]:
viz.Boxplot(MPE)

## Histograms

In [ ]:
viz.Histograms(MPE)

In [ ]:
f = '/home/b3053674/Documents/pycotools/ZiModel/data.csv'
f2 = '/home/b3053674/Documents/pycotools/ZiModel/data2.csv'

df = pandas.read_csv(f)
# df = df.drop(u'Unnamed: 0')
df = df.sort_values(by=['Time', 'ParameterFitIndex'])
# import seaborn
df.to_csv(f2)
# seaborn.tsplot(df, time='Time',value='Smad3c', unit='ParameterFitIndex')

In [ ]:
gammas = seaborn.load_dataset("gammas").head()

In [ ]:
ax = sns.tsplot(time="timepoint", value="BOLD signal",
                unit="subject", condition="ROI",
                data=gammas)

To monitor progress, look either at the task manager or system monitor. 